In [6]:
import numpy as np

# Specify the path to the CSV file
file_path = r"C:\Users\kubaw\Desktop\DELFT\THESIS\CH6\battery_Cost_EUR.csv"

df = pd.read_csv(file_path)

pv_cost_arr = df.to_numpy()
pv_cost_arr

array([1.0502 , 0.7758 , 0.65974, 0.58734, 0.52177, 0.4041 , 0.31188,
       0.22885, 0.17893, 0.16342, 0.14032, 0.12675, 0.1478 , 0.12649])

In [11]:
from GBM_FOR_ELECTRICITY_TARIFF import generate_GBM_from_data
import pandas as pd

In [12]:
n_paths = 10000
nsteps = 25
total_time = 25
xzero = 1

In [13]:
simulation_data, mu, sigma = generate_GBM_from_data(pv_cost_arr, n_paths, nsteps, total_time, xzero)
simulation_df = pd.DataFrame(simulation_data)

sigma

0.12224338907662426

In [26]:
initial_price = 1
initial_capacity = 6
final_capacity = 147
years = 25
learning_rate = 0.17

In [30]:
import numpy as np

growth_factor = (final_capacity / initial_capacity) ** (1 / years)

cumulative_capacities = [initial_capacity * (growth_factor ** year) for year in range(years + 1)]

prices = [initial_price]
for i in range(1, len(cumulative_capacities)):
    if cumulative_capacities[i - 1] > 0:
        doubling_ratio = cumulative_capacities[i] / cumulative_capacities[i - 1]
    else:
        doubling_ratio = 0
    
    num_doublings = np.log2(doubling_ratio)
    
    price_decrease_factor = (1 - learning_rate) ** num_doublings
    
    current_price = prices[-1] * price_decrease_factor
    
    prices.append(current_price)

initial_doublings = np.log2(cumulative_capacities[1] / initial_capacity)
prices[0] = initial_price * ((1 - learning_rate) ** initial_doublings)

yearly_prices = {f"Year {i}": round(price, 2) for i, price in enumerate(prices)}
yearly_prices_list = list(yearly_prices.values())
yearly_prices_arr = np.array(yearly_prices_list) / 1000
yearly_prices_arr

array([0.00097, 0.00097, 0.00093, 0.0009 , 0.00087, 0.00084, 0.00081,
       0.00079, 0.00076, 0.00073, 0.00071, 0.00068, 0.00066, 0.00064,
       0.00062, 0.0006 , 0.00058, 0.00056, 0.00054, 0.00052, 0.0005 ,
       0.00049, 0.00047, 0.00045, 0.00044, 0.00042])

In [34]:
# The parameters for the simulation
n_paths = 10000
nsteps = 50
total_time = 50
xzero = 0.3

simulation_data, mu, sigma = generate_GBM_from_data(yearly_prices_arr, n_paths, nsteps, total_time, xzero)
simulation_df = pd.DataFrame(simulation_data)

mu

-0.03831141250548132

In [55]:
import numpy as np

# Parameters for the geometric Brownian motion
timesteps = 25  # Number of timesteps
mu = -0.038  # Drift coefficient
sigma = 0.122  # Volatility coefficient
S0 = 1  # Initial value of the process

# Time step (assuming each time step is 1 unit of time)
dt = 1
n_of_sim = 1000

simualations = []

# Run the simulations
for s in range(n_of_sim):
    # Simulate random changes using the normal distribution
    random_changes = np.random.normal(loc=(mu - (sigma**2)/2)*dt, scale=sigma*np.sqrt(dt), size=timesteps)

    # Calculate the geometric Brownian motion
    geometric_brownian_motion = [S0] + list(S0 * np.exp(np.cumsum(random_changes)))

    # Append the simulation to the list
    simualations.append(geometric_brownian_motion)

simualations

[[1,
  0.8557150518092591,
  0.7893855858940373,
  0.737909973547026,
  0.7522533117379316,
  0.8493129101153835,
  0.6482885795843489,
  0.6214953976849938,
  0.6695564422154919,
  0.5829503770382902,
  0.589687206797483,
  0.6617435232730617,
  0.5996536708516405,
  0.5259489683512497,
  0.5467442513897952,
  0.4606248029453464,
  0.366983225727154,
  0.4057946850792319,
  0.38432601900910235,
  0.36945428167053135,
  0.3806374206376549,
  0.41233660666086913,
  0.43801179917444066,
  0.43327405190672014,
  0.47875226463218956,
  0.5435224395036741],
 [1,
  0.8868299091365849,
  0.9464155477770648,
  1.0156478458948566,
  0.893860175969805,
  0.9554761914246157,
  0.9243016132729062,
  0.9487884346527559,
  0.9750205563899561,
  1.3322067831219166,
  1.435740057771153,
  1.0620424387969867,
  1.0661477533679393,
  0.8289790676133838,
  0.653090539329361,
  0.5426137815862645,
  0.5892253571176634,
  0.5170745185374508,
  0.5058697690709111,
  0.4327004548551206,
  0.42728795003308256

In [56]:
import plotly.graph_objs as go
import numpy as np

# Create a line plot using Plotly for each numpy array
traces = [go.Scatter(
    x=np.arange(len(array)),  # Adjusting the x-axis to the length of the array
    y=array,  # Y-axis values, one of the arrays from the list
    mode='lines',
    line=dict(width=1),
    opacity=0.5  # semi-transparency to see overlapping lines
) for array in simualations]

# Define the layout of the plot
layout = go.Layout(
    title="CAPEX Development Over Time",
    xaxis=dict(title='Timestep'),
    yaxis=dict(title='CAPEX (eur/Wp)'),
    showlegend=False,
    plot_bgcolor='#f2f1ee',
    paper_bgcolor='white'
)

# Create the figure with all the traces and the specified layout
fig = go.Figure(data=traces, layout=layout)

# Update the background colors after creating the figure
fig.update_layout(
    plot_bgcolor='#f2f1ee',
    paper_bgcolor='white'
)

# Show the figure
fig.show()

In [39]:
# Convert the list of numpy arrays into a DataFrame
import pandas as pd

batery_CAPES = pd.DataFrame(simualations)
batery_CAPES = batery_CAPES.T

In [40]:
file_path = r'C:\Users\kubaw\Desktop\DELFT\THESIS\CH6\batery_CAPES.csv'
batery_CAPES.to_csv(file_path, index=False)

In [46]:
import plotly.express as px
import pandas as pd

pv_cost_df = pd.DataFrame(pv_cost_arr)
pv_cost_df.rename(columns={0: "Price"})

years_list_corrected = list(pv_cost_eur.keys())
years_list_df = pd.DataFrame(years_list_corrected)

years_list_df.rename(columns={0: "Year"})


# Applying column names directly during DataFrame creation
combined_df_corrected = pd.DataFrame({
    "Year": years_list_corrected,
    "Price": pv_cost_arr
})
combined_df_corrected

,Year,Price
0,2010,1.05020
1,2011,0.77580
2,2012,0.65974
3,2013,0.58734
4,2014,0.52177
5,2015,0.40410
6,2016,0.31188
7,2017,0.22885
8,2018,0.17893
9,2019,0.16342


In [48]:


# Create the graph using plotly.express with 'Step' on the x-axis and 'Value' on the y-axis
fig = px.line(combined_df_corrected, x = 'Year', y ='Price', title='Lithium-Ion Batery Cost',
              labels={'Year':'Year', 'Price':'CAPEX (eur/kW)'})

fig.update_traces(line=dict(color='#fc8b9b', width=3), mode='markers+lines', marker=dict(size=10),)

fig.update_layout(
    plot_bgcolor='#f2f1ee',  # Set the background color to white
    paper_bgcolor='white',  # Set the background color outside the plot to white
    xaxis=dict(
        showline=True,
        showgrid=True,
        gridcolor='white',  # Set grid color to light grey
        linecolor='white',
    ),
    yaxis=dict(
        showline=True,
        showgrid=True,
        gridcolor='white',  # Set grid color to light grey
        linecolor='white',
    )
)

# Update the font size
fig.update_layout(
    title_font_size=14,
    legend_title_font_size=14,
    width = 1100,
    height = 700
)


# Show the plot
every_second_year = combined_df_corrected['Year'][::2]

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=every_second_year,
        ticktext=every_second_year
    ),
)

fig.show()